In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression, LogisticRegression
import warnings
warnings.filterwarnings('ignore')
import pprint
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

## 1. DATA PREPERATION

In [16]:
proj_data = pd.read_csv('application_data.csv')
proj_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


### DROPPING COLUMNS


In [17]:
columns_dropped = ['NAME_CONTRACT_TYPE','CODE_GENDER','CNT_CHILDREN','NAME_TYPE_SUITE','FLAG_MOBIL','FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL','WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION','LIVE_REGION_NOT_WORK_REGION','REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY','APARTMENTS_AVG','BASEMENTAREA_AVG','YEARS_BEGINEXPLUATATION_AVG','YEARS_BUILD_AVG','COMMONAREA_AVG','ELEVATORS_AVG','ENTRANCES_AVG','FLOORSMAX_AVG','FLOORSMIN_AVG','LANDAREA_AVG','LIVINGAPARTMENTS_AVG','LIVINGAREA_AVG','NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_AVG','APARTMENTS_MODE','BASEMENTAREA_MODE','YEARS_BEGINEXPLUATATION_MODE','YEARS_BUILD_MODE','COMMONAREA_MODE','ELEVATORS_MODE','ENTRANCES_MODE','FLOORSMAX_MODE','FLOORSMIN_MODE','LANDAREA_MODE','LIVINGAPARTMENTS_MODE','LIVINGAREA_MODE','NONLIVINGAPARTMENTS_MODE','NONLIVINGAREA_MODE','APARTMENTS_MEDI','BASEMENTAREA_MEDI','YEARS_BEGINEXPLUATATION_MEDI','YEARS_BUILD_MEDI','COMMONAREA_MEDI','ELEVATORS_MEDI','ENTRANCES_MEDI','FLOORSMAX_MEDI','FLOORSMIN_MEDI','LANDAREA_MEDI','LIVINGAPARTMENTS_MEDI','LIVINGAREA_MEDI','NONLIVINGAPARTMENTS_MEDI','NONLIVINGAREA_MEDI','FONDKAPREMONT_MODE','HOUSETYPE_MODE','TOTALAREA_MODE','WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE','DAYS_LAST_PHONE_CHANGE','FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21','AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']

processed_data = proj_data.drop(columns_dropped, axis=1)
processed_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 30 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SK_ID_CURR                   307511 non-null  int64  
 1   TARGET                       307511 non-null  int64  
 2   FLAG_OWN_CAR                 307511 non-null  object 
 3   FLAG_OWN_REALTY              307511 non-null  object 
 4   AMT_INCOME_TOTAL             307511 non-null  float64
 5   AMT_CREDIT                   307511 non-null  float64
 6   AMT_ANNUITY                  307499 non-null  float64
 7   AMT_GOODS_PRICE              307233 non-null  float64
 8   NAME_INCOME_TYPE             307511 non-null  object 
 9   NAME_EDUCATION_TYPE          307511 non-null  object 
 10  NAME_FAMILY_STATUS           307511 non-null  object 
 11  NAME_HOUSING_TYPE            307511 non-null  object 
 12  REGION_POPULATION_RELATIVE   307511 non-null  float64
 13 

### DATA CLEANING 

#### Reformat to AGE

In [18]:
# Data Cleaning
# Check for null values
processed_data['AGE'] = processed_data['DAYS_BIRTH']/-365
processed_data['AGE'] = processed_data['AGE'].astype(int)


### FILLING NA DATA

In [19]:
numerical_cols = processed_data.select_dtypes(include=['float', 'int']).columns
processed_data[numerical_cols] = processed_data[numerical_cols].fillna(processed_data[numerical_cols].median())
processed_data.fillna('Missing', inplace=True)
processed_data.describe()


,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,AGE
count,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307511.000000,3.075110e+05,307511.000000,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,3.075110e+05,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000
mean,278180.518577,0.080729,1.687979e+05,5.990260e+05,27108.487841,5.383163e+05,0.020868,-16036.995067,63815.045904,-4986.120328,...,2.052463,2.031521,0.504311,5.145034e-01,0.515695,1.417523,0.142944,1.400626,0.099717,43.435968
std,102790.175348,0.272419,2.371231e+05,4.024908e+05,14493.461065,3.692890e+05,0.013831,4363.988632,141275.766519,3522.886321,...,0.509034,0.502737,0.139408,1.908699e-01,0.174736,2.398395,0.446033,2.377224,0.361735,11.954593
min,100002.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,-24672.000000,...,1.000000,1.000000,0.014568,8.173617e-08,0.000527,0.000000,0.000000,0.000000,0.000000,20.000000
25%,189145.500000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,-7479.500000,...,2.000000,2.000000,0.505998,3.929737e-01,0.417100,0.000000,0.000000,0.000000,0.000000,34.000000
50%,278202.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,-4504.000000,...,2.000000,2.000000,0.505998,5.659614e-01,0.535276,0.000000,0.000000,0.000000,0.000000,43.000000
75%,367142.500000,0.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,-2010.000000,...,2.000000,2.000000,0.505998,6.634218e-01,0.636376,2.000000,0.000000,2.000000,0.000000,53.000000
max,456255.000000,1.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,0.000000,...,3.000000,3.000000,0.962693,8.549997e-01,0.896010,348.000000,34.000000,344.000000,24.000000,69.000000


In [11]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 30 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SK_ID_CURR                   307511 non-null  int64  
 1   TARGET                       307511 non-null  int64  
 2   FLAG_OWN_CAR                 307511 non-null  object 
 3   FLAG_OWN_REALTY              307511 non-null  object 
 4   AMT_INCOME_TOTAL             307511 non-null  float64
 5   AMT_CREDIT                   307511 non-null  float64
 6   AMT_ANNUITY                  307511 non-null  float64
 7   AMT_GOODS_PRICE              307511 non-null  float64
 8   NAME_INCOME_TYPE             307511 non-null  object 
 9   NAME_EDUCATION_TYPE          307511 non-null  object 
 10  NAME_FAMILY_STATUS           307511 non-null  object 
 11  NAME_HOUSING_TYPE            307511 non-null  object 
 12  REGION_POPULATION_RELATIVE   307511 non-null  float64
 13 

### FILTER BY SELF_EMPLOYEED

In [27]:
filtered_data = processed_data[processed_data['ORGANIZATION_TYPE'] == 'Self-employed']
filtered_data.describe()

,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,AGE
count,38412.000000,38412.000000,3.841200e+04,3.841200e+04,38412.000000,3.841200e+04,38412.000000,38412.000000,38412.000000,38412.000000,...,38412.00000,38412.000000,38412.000000,38412.000000,38412.000000,38412.000000,38412.000000,38412.000000,38412.000000,38412.000000
mean,278947.616396,0.101739,1.674424e+05,5.890418e+05,27807.360708,5.278394e+05,0.018959,-14547.136832,-1957.712199,-4500.203869,...,2.11374,2.096767,0.488193,0.501349,0.514593,1.470009,0.142976,1.451968,0.099136,39.355878
std,103118.225412,0.302309,9.429796e+04,3.769039e+05,14312.110682,3.441923e+05,0.011526,3419.030343,1536.577092,3148.782216,...,0.45523,0.447130,0.138364,0.190876,0.169668,2.332875,0.441017,2.312914,0.356381,9.366335
min,100017.000000,0.000000,2.700000e+04,4.500000e+04,2317.500000,4.050000e+04,0.000533,-25168.000000,-16360.000000,-20841.000000,...,1.00000,1.000000,0.015600,0.000005,0.000527,0.000000,0.000000,0.000000,0.000000,20.000000
25%,189191.750000,0.000000,1.125000e+05,2.710665e+05,17167.500000,2.430000e+05,0.009334,-17115.000000,-2726.000000,-6715.000000,...,2.00000,2.000000,0.477680,0.376882,0.424130,0.000000,0.000000,0.000000,0.000000,32.000000
50%,279612.000000,0.000000,1.485000e+05,5.123385e+05,26086.500000,4.500000e+05,0.018634,-14382.000000,-1575.000000,-4138.500000,...,2.00000,2.000000,0.505998,0.551960,0.535276,0.000000,0.000000,0.000000,0.000000,39.000000
75%,368781.250000,0.000000,2.025000e+05,8.086500e+05,35617.500000,6.795000e+05,0.026392,-11770.750000,-823.000000,-1822.000000,...,2.00000,2.000000,0.505998,0.650536,0.622922,2.000000,0.000000,2.000000,0.000000,46.000000
max,456248.000000,1.000000,4.500000e+06,2.695500e+06,225000.000000,2.475000e+06,0.072508,-7489.000000,0.000000,0.000000,...,3.00000,3.000000,0.938832,0.855000,0.893976,23.000000,6.000000,23.000000,5.000000,68.000000


### EXPORT TO CSV

In [28]:
filtered_data.to_csv("processed_data.csv")